<a href="https://colab.research.google.com/github/Marcelino369/Actividad14/blob/main/Actividad14_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, corr


spark = SparkSession.builder.appName("Semana14-2").getOrCreate()


In [ ]:
spark

In [ ]:
# Cargamos el archivo CSV
df = spark.read.csv("Supermarket_Sales.csv", header=True, inferSchema=True)

df.show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|2024-11-13 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|2024-11-13 10:29:00|       Cash|  76.4|            4.761904762|      

## MEDIA:
Es el promedio de un conjunto de datos. Se calcula sumando todos los valores y dividiendo entre la cantidad de elementos.

In [ ]:
# Filtramos por una fecha especifica(Date) y una línea de producto(Product Line)
# Para Obtener: El promedio o LA MEDIA de ventas que tuvo

fecha_especifica = '1/1/2019'
linea_producto = 'Electronic accessories'

# Filtrar el DataFrame por fecha y línea de producto
df_filtrado = df.filter((df["Date"] == fecha_especifica) & (df["Product line"] == linea_producto))
df_filtrado.show()
# Calcular la media del precio unitario de nuestro *df_filtrado
media = df_filtrado.select(mean(col("Unit price")).alias("Media")).collect()[0]["Media"]

# Resultado
print(f"Media del precio unitario para '{linea_producto}' en la fecha {fecha_especifica}: {media}")

+-----------+------+---------+-------------+------+--------------------+----------+--------+------+-------+--------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity|Tax 5%|  Total|    Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+------+-------+--------+-------------------+-----------+------+-----------------------+------------+------+
|891-01-7034|     B| Mandalay|       Normal|Female|Electronic access...|     74.71|       6|22.413|470.673|1/1/2019|2024-11-13 19:07:00|       Cash|448.26|            4.761904762|      22.413|   6.7|
|556-97-7101|     C|Naypyitaw|       Normal|Female|Electronic access...|     63.22|       2| 6.322|132.762|1/1/2019|2024-11-13 15:51:00|       Cash|126.44|            4.761904762|       6.322|   8.5|


# Moda:
Es el valor que más se repite en un conjunto de datos. Si no hay valores repetidos, se dice que no hay moda.

In [ ]:
# Filtramos por una sede(Branch) y una linea de Producto(Product Line)
# Para Obtener: La cantidad de productos comprados (Quantity)

sede = 'B'
linea_producto = 'Fashion accessories'

# Filtrar el DataFrame por sede y línea de producto
df_filtrado = df.filter((df["Branch"] == sede) & (df["Product line"] == linea_producto))
df_filtrado.show()
# Calcular la moda de la cantidad de productos que compran por sede en cierta sede *df_filtrado
moda = df_filtrado.groupBy("Quantity").count().orderBy("count", ascending=False).first()["Quantity"]

# Resultado
print(f"La linea de Producto '{linea_producto}' en la sede {sede}: {moda}")

+-----------+------+--------+-------------+------+-------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|    City|Customer type|Gender|       Product line|Unit price|Quantity| Tax 5%|   Total|     Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+--------+-------------+------+-------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|351-62-0822|     B|Mandalay|       Member|Female|Fashion accessories|     14.48|       4|  2.896|  60.816| 2/6/2019|2024-11-13 18:07:00|    Ewallet| 57.92|            4.761904762|       2.896|   4.5|
|649-29-6775|     B|Mandalay|       Normal|  Male|Fashion accessories|     33.52|       1|  1.676|  35.196| 2/8/2019|2024-11-13 15:31:00|       Cash| 33.52|            4.761904762|       1.676|   

# Mediana:
Es el valor que está en el centro de un conjunto de datos ordenados.


In [ ]:
# Filtramos por un metodo de pago(Payment) y la cantidad de productos comprados(Quantity)
# Para Obtener: Un promedio del precio total que compran con ciertos metodos de pago

pago = 'Credit card'
cantidad = '10'

# Filtrar el DataFrame por sede y línea de producto
df_filtrado = df.filter((df["Payment"] == pago) & (df["Quantity"] == cantidad))
df_filtrado.show()
# Calcular la mediana del total que pagan los clientes *df_filtrado
percentage = df_filtrado.approxQuantile("Total", [0.5], 0.0)
mediana = percentage

# Resultado
print(f"El clientes pagan con '{pago}' para comprar {cantidad} cantidad: ${mediana}")

+-----------+------+---------+-------------+------+--------------------+----------+--------+------+--------+---------+-------------------+-----------+-----+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity|Tax 5%|   Total|     Date|               Time|    Payment| cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+------+--------+---------+-------------------+-----------+-----+-----------------------+------------+------+
|199-75-8169|     A|   Yangon|       Member|  Male|   Sports and travel|     15.81|      10| 7.905| 166.005| 3/6/2019|2024-11-13 12:27:00|Credit card|158.1|            4.761904762|       7.905|   8.6|
|393-65-2792|     C|Naypyitaw|       Normal|  Male|  Food and beverages|     89.48|      10| 44.74|  939.54| 1/6/2019|2024-11-13 12:46:00|Credit card|894.8|            4.761904762|       44.74|   

**Correlación:**
Mide la relación entre dos variables. Indica si hay una relación directa (positiva), inversa (negativa) o si no existe ninguna (cero). Se mide entre -1 y 1, donde valores cercanos a 1 o -1 indican una relación fuerte y cercanos a 0, una relación débil.

In [113]:
# Filtramos por linea de producto (Product Line) y perido de tiempo (><Date)
# Para Obtener: una relacion entre las variables del precio unitario y la cantidad
fecha = '3/30/2019'

linea1 = 'Food and beverages'

# Filtrar el DataFrame por sede y línea de producto
df_filtrado = df.filter((df["Date"] > fecha) & (df["Product line"] == linea1))
df_filtrado.show()

# Calcular la correlación entre 'Unit price' y 'Quantity' para df_filtrado1
correlation = df_filtrado.select(corr(df["Unit price"], df["Quantity"]).alias("correlation")).collect()[0]["correlation"]

# Resultados
print(f"Correlación entre 'Unit price' y 'Quantity' para la {fecha} en adelante: {correlation}")

+-----------+------+---------+-------------+------+------------------+----------+--------+-------+--------+--------+-------------------+-------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|      Product line|Unit price|Quantity| Tax 5%|   Total|    Date|               Time|Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+------------------+----------+--------+-------+--------+--------+-------------------+-------+------+-----------------------+------------+------+
|228-96-1411|     C|Naypyitaw|       Member|Female|Food and beverages|      98.7|       8|  39.48|  829.08|3/4/2019|2024-11-13 20:39:00|   Cash| 789.6|            4.761904762|       39.48|   7.6|
|510-95-6347|     B| Mandalay|       Member|Female|Food and beverages|     48.52|       3|  7.278| 152.838|3/5/2019|2024-11-13 18:17:00|Ewallet|145.56|            4.761904762|       7.278|   4.0|
|829-49-1914|     C|

In [ ]:
spark.stop()